In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
updated_FRL = pd.read_excel("/share/data/school_choice/dssg/riyab_FRL_data.xlsx", None, engine='openpyxl')

In [ ]:
grouped_Geoid = updated_FRL["Grouped GeoID External"]
grouped_Geoid.tail(10)

In [ ]:
#filtering out blocks with less than 5 students for now (GEOID group is not the GEOID)
grouped_Geoid_filtered = grouped_Geoid[grouped_Geoid["Geoid Group"].astype('str').str.len() > 3]
grouped_Geoid_filtered

In [ ]:
df = pd.read_csv("~/displacement-typologies/data/outputs/typologies/SanFrancisco_typology_output.csv", sep=",")
df["GEOID"]

In [ ]:
grouped_Geoid_filtered["Geoid Group"]

Getting the first 10 digits of the GEOID group to access the [Census Tract id](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html). This allows us to merge the FRL data Henry shared with us with the block database. 

In [ ]:
grouped_Geoid_filtered["New Geoid"] = grouped_Geoid_filtered["Geoid Group"].astype(str).str[:10].astype(int)

In [ ]:
grouped_Geoid_filtered

In [ ]:
grouped_Geoid_filtered.dtypes

In [ ]:
grouped_Geoid_filtered = grouped_Geoid_filtered.merge(df, left_on = "New Geoid", right_on = "GEOID")

In [ ]:
grouped_Geoid_new = grouped_Geoid_filtered[["GEOID", "4YR AVG Student Count", "4YR AVG FRL Count", "4YR AVG Eth Flag Count", "4YR AVG Combo Flag Count",
                                            "OD", "ARG", "EOG", "AdvG", "SMMI", "ARE", "BE", "SAE"]]
grouped_Geoid_new

In [ ]:
grouped_Geoid_new = grouped_Geoid_new.drop_duplicates()

In [ ]:
grouped_Geoid_new.head(20)

In [ ]:
grouped_Geoid_new = grouped_Geoid_new.dropna(axis=0, how='all')
grouped_Geoid_new = grouped_Geoid_new.dropna(axis=1, how='all')

In [ ]:
grouped_Geoid_new = grouped_Geoid_new.groupby("GEOID").sum()
grouped_Geoid_new

In [ ]:
grouped_Geoid_new
#write a script to create a new "gentrification column"

Wrote a script through run through each of the gentrification columns, and find the categorical value for each row - which will then be added to the `gentrification_values` list

In [ ]:
gentrification_cols = ["OD", "ARG", "EOG", "AdvG", "SMMI", "ARE", "BE", "SAE"]
gentrification_values = []

for i in grouped_Geoid_new.index: 
    new_value = None
    for col in gentrification_cols: 
        if grouped_Geoid_new.loc[i, col] != 0.0: 
            new_value = col
    gentrification_values.append(new_value)
len(grouped_Geoid_new)

In [ ]:
len(gentrification_values)

Filter out rows that have "None" in their gentrification column

In [ ]:
grouped_Geoid_new["Gentrification"] = gentrification_values
grouped_Geoid_new.head(30)
grouped_Geoid_gent = grouped_Geoid_new[grouped_Geoid_new["Gentrification"].notnull()]
grouped_Geoid_gent.head(10)

In [ ]:
grouped_Geoid_gent.plot()

In [ ]:
grouped_Geoid_gent = grouped_Geoid_gent[["4YR AVG Student Count", "4YR AVG FRL Count", "4YR AVG Eth Flag Count", "4YR AVG Combo Flag Count", "Gentrification"]]
grouped_Geoid_gent.head(10)

In [ ]:
gentrification_agg = grouped_Geoid_gent.groupby("Gentrification").mean()
gentrification_agg_new = gentrification_agg.drop("4YR AVG Student Count", axis = 1)
gentrification_agg_new.loc[gentrification_cols].plot.bar()

This grouped bar chart illustrates the distribution of various 4YR demographic counts in terms of gentrification typology. 

In [ ]:
gentrification_agg

In [ ]:
labels = ["Student", "FRL", "AALPI", "Combo"]
plt.bar(gentrification_cols, gentrification_agg["4YR AVG Student Count"])
plt.bar(gentrification_cols, gentrification_agg["4YR AVG FRL Count"])
plt.bar(gentrification_cols, gentrification_agg["4YR AVG Eth Flag Count"])
plt.bar(gentrification_cols, gentrification_agg["4YR AVG Combo Flag Count"])
plt.legend(labels)

This is a stacked barchart illustrating the counts of each demographic group in blocks compared to the block's gentrification status. We can see the breakdown of demographic factors within each gentrification status. 

In [ ]:
gentrification_agg_new = gentrification_agg_new.reindex(gentrification_cols)
gentrification_agg_new

In [ ]:
x_axis_labels = ["FRL", "AALPI", "Combo"]
sns.heatmap(gentrification_agg_new, xticklabels = x_axis_labels)

This chart shows us that OD, EOG, and AdvG hold the highest proportion of focal students (want to target those communities). We can see also that on the lower end of the y_axis, or higher on the gentrification spectrum, there are less focal students.

In [ ]:
grouped_Geoid_new.loc[1, "ARE"]

In [ ]:
#heatmap of FRL/AALPI vs CTIP assignments would also be interesting 
block_df_dict = pd.read_excel("/share/data/school_choice/Data/SF 2010 blks 022119 with field descriptions (1).xlsx", None, engine='openpyxl')

In [ ]:
block_database = block_df_dict["block database"]
block_database = block_database.dropna(axis=0, how='all')
block_database = block_database.dropna(axis=1, how='all')
block_database["CTIP_2013 assignment"]

In [ ]:
ctip_vs_demo = block_database.merge(grouped_Geoid_filtered, left_on = "Tract", right_on = "GEOID")
ctip_vs_demo

In [ ]:
ctip_vs_demo = ctip_vs_demo[["GEOID", "CTIP_2013 assignment", "OD", "ARG", "EOG", "AdvG", "SMMI", "ARE", "BE", "SAE"]]
ctip_vs_demo

In [ ]:
ctip_vs_demo